In [1]:
from kmodes.kprototypes import KPrototypes
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import itertools


In [2]:
path = '/Users/mehmetdeniz/Library/CloudStorage/GoogleDrive-mehmetbakideniz@gmail.com/My Drive/meta_audience_insights_age_data.csv'
df=pd.read_csv(path)

In [3]:
df.head()

,id,schedule_id,page_view,engagement,all_leads,leads,install,purchase,add_to_cart,signup,begin_checkout,unspecified,page_view_value,all_leads_value,leads_value,install_value,purchase_value,add_to_cart_value,signup_value,begin_checkout_value,unspecified_value,campaign_budget,campaign_status,campaign_name,clicks,cpc,cpm,cpp,created_time,ctr,impressions,campaign_type,optimization_strategy,reach,social_spend,cost,gender,age,cpi,cr,roas,date_interval
0,175455,901,7815.0,10691.0,0.0,0.0,0.0,20.0,108.0,0.0,163.0,0.0,38961.89,0.0,0.0,0.0,50361.23,304992.02,0.0,758350.52,0.0,NaN,NaN,dpa_ig_prospecting_sneaker,11771.0,0.49445,11.96777,NaN,NaN,0.02420,486323.0,PRODUCT_CATALOG_SALES,Unknown Optimization Goal,225782.0,NaN,5820.20,female,18-24,NaN,0.00170,8.65283,56
1,175456,901,6394.0,8131.0,0.0,0.0,0.0,12.0,71.0,0.0,117.0,0.0,28090.49,0.0,0.0,0.0,38238.21,193903.68,0.0,969837.36,0.0,NaN,NaN,dpa_ig_prospecting_sneaker,8800.0,0.45893,11.64267,NaN,NaN,0.02537,346874.0,PRODUCT_CATALOG_SALES,Unknown Optimization Goal,136954.0,NaN,4038.54,male,18-24,NaN,0.00136,9.46833,56
2,175457,901,70.0,130.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,619.97,0.0,0.0,0.0,0.00,14798.10,0.0,7999.90,0.0,NaN,NaN,dpa_ig_prospecting_sneaker,136.0,0.52434,14.06786,NaN,NaN,0.02683,5069.0,PRODUCT_CATALOG_SALES,Unknown Optimization Goal,1792.0,NaN,71.31,unknown,18-24,NaN,0.00000,0.00000,56
3,175458,901,8752.0,12094.0,0.0,0.0,0.0,14.0,71.0,0.0,132.0,0.0,26627.10,0.0,0.0,0.0,41088.52,214216.45,0.0,726487.30,0.0,NaN,NaN,dpa_ig_prospecting_sneaker,13246.0,0.50062,16.20660,NaN,NaN,0.03237,409171.0,PRODUCT_CATALOG_SALES,Unknown Optimization Goal,197111.0,NaN,6631.27,female,25-34,NaN,0.00106,6.19618,56
4,175459,901,10642.0,13677.0,0.0,0.0,0.0,13.0,62.0,0.0,123.0,0.0,30167.35,0.0,0.0,0.0,38577.77,189660.10,0.0,778741.02,0.0,NaN,NaN,dpa_ig_prospecting_sneaker,14547.0,0.47470,16.50060,NaN,NaN,0.03476,418498.0,PRODUCT_CATALOG_SALES,Unknown Optimization Goal,189943.0,NaN,6905.47,male,25-34,NaN,0.00089,5.58655,56


In [4]:

# Define the columns for grouping and aggregation
columns = ['gender', 'age', 'campaign_type', 'optimization_strategy']
aggregations = {'cost': 'sum', 'clicks': 'sum', 'purchase_value': 'sum'}

# Initialize an empty list to store the grouped DataFrames
grouped_dfs = []

# Perform grouping operations for different combinations of columns
for i in range(1, len(columns) + 1):
    combinations = itertools.combinations(columns, i)
    for combo in combinations:
        grouped_df = df.groupby(list(combo)).agg(aggregations).reset_index()
        #grouped_df['combinations'] = grouped_df[combo].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

        grouped_dfs.append(grouped_df)
        
df2 = pd.concat(grouped_dfs, axis=0, join='outer')

# merged_df will contain the final merged DataFrame

In [5]:
df2.head()

,gender,cost,clicks,purchase_value,age,campaign_type,optimization_strategy
0,female,946650.03348,1371121.0,9438829.50,NaN,NaN,NaN
1,male,906833.70359,1137256.0,9997000.03,NaN,NaN,NaN
2,unknown,18259.13359,21788.0,263759.36,NaN,NaN,NaN
0,NaN,646356.19327,878412.0,7411329.39,18-24,NaN,NaN
1,NaN,641755.16039,890027.0,7164112.44,25-34,NaN,NaN


In [6]:
df2['cpc'] = np.where(df2['clicks'] != 0, df2['cost'] / df2['clicks'], 0)
df2['roas']=np.where(df2['cost'] != 0, df2['purchase_value'] / df2['cost'], 0)
df2.drop(['clicks', 'purchase_value', 'cost'], axis=1, inplace= True )

In [7]:
df2.head()

,gender,age,campaign_type,optimization_strategy,cpc,roas
0,female,NaN,NaN,NaN,0.690420,9.970770
1,male,NaN,NaN,NaN,0.797387,11.024072
2,unknown,NaN,NaN,NaN,0.838036,14.445338
0,NaN,18-24,NaN,NaN,0.735824,11.466324
1,NaN,25-34,NaN,NaN,0.721051,11.163311


In [8]:
df2.reset_index(inplace=True, drop=True)

In [9]:
import pandas as pd

# Assuming df2 is your DataFrame
columns = ['gender', 'age', 'campaign_type', 'optimization_strategy']

# Create the 'combinations' column
df2['combinations'] = df2[columns].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)

# Drop the original columns
df2 = df2.drop(columns=columns)

In [10]:
df2.drop(['index'], axis=1, inplace= True )


KeyError: "['index'] not found in axis"

In [11]:
from sklearn.ensemble import IsolationForest

# Select the columns of interest
columns_of_interest = ['roas', 'cpc']

# Create a new DataFrame with only the columns of interest
df_subset = df2[columns_of_interest]

# Create an instance of the IsolationForest model
isolation_forest = IsolationForest(n_estimators=100,max_samples='auto',contamination=float(0.1))

# Fit the model on the data
isolation_forest.fit(df_subset)

# Predict anomalies for the data points
predictions = isolation_forest.predict(df_subset)

# Add the "IsAnomaly" column to df2
df2['IsAnomaly'] = predictions

In [12]:
path = '/Users/mehmetdeniz/Library/CloudStorage/GoogleDrive-mehmetbakideniz@gmail.com/My Drive/result_ISF2.csv'

df2[df2['IsAnomaly'] ==-1].to_csv(path)

In [13]:
df2['IsAnomaly'].value_counts()

IsAnomaly
 1    267
-1     30
Name: count, dtype: int64

In [14]:
print('hello')

hello
